In [1]:
import requests,re,time
import pandas as pd

In [2]:
https://www.amap.com/dir?from%5Badcode%5D=441702&from%5Bname%5D=%E9%98%B3%E6%B1%9F%E7%AB%99&from%5Bid%5D=B0FFJ0EEBX&from%5Bpoitype%5D=150200&from%5Blnglat%5D=111.99019599999997%2C21.810437&from%5Bmodxy%5D=111.989324%2C21.809852&to%5Bid%5D=B030000DPY-to&to%5Bname%5D=%E6%BD%AE%E5%B7%9E%E7%AB%99&to%5Blnglat%5D=116.584079%2C23.638136&to%5Bmodxy%5D=116.58464%2C23.638122&to%5Bpoitype%5D=150200&to%5Badcode%5D=445103&type=bus&policy=0&dateTime=now
key= '4cf31868da639ba20f04d33ca5218dff' #'6a803535ccaeb1ad3677a7ec2bf80c00'
locationdic={}
def getdic(url):
    count=0
    while True :
        try:
            res = requests.get(url)
            dic = res.json()

            return dic
        except:
            time.sleep(1)
            count+=1

In [3]:
def getxy(address):
    try:
        if address in locationdic:
            return locationdic[address]
        url=f'https://restapi.amap.com/v3/place/text?keywords={address}&offset=20&page=1&key={key}&citylimit=true'
        #res = requests.get(url)
        dic = getdic(url)
        pois=dic['pois']
        location=pois[0]['location']
        locationdic[address]=location
        return location
    except Exception as e:
        print(e)

In [4]:
def getdrive(s,e):

    url=f'https://restapi.amap.com/v3/direction/driving?origin={s}&destination={e}&key={key}'
    # res = requests.get(url)
    dic = getdic(url)
    route=dic['route']['paths']
    distance=float(route[0]['distance'])
    duration=float(route[0]['duration'])/60
    return  distance,duration

try:
    last_dis_df=pd.read_excel('distance.xlsx')
    last_dur_df = pd.read_excel('duration.xlsx')
    print('读取原始数据成功')
except:
    print('读取原始数据失败')

读取原始数据失败


In [5]:
places=['广州市越秀区府前路1号市政府大院', '深圳市民中心c区', '珠海市香洲区人民东路2号', '汕头市金平区跃进路28号', '佛山市禅城区岭南大道北12号', '韶关市浈江区风度北路75号', '河源市源城区沿江中路19号', '梅州市梅江区新中路38号', '惠州市惠城区云山西路6号', '汕尾市城区汕尾大道北市政府办公楼', '东莞市南城胜和鸿福路99号', '中山市松苑路1号', '江门市蓬江区白沙大道西1号', '阳江市江城区东风二路60号', '湛江市赤坎区跃进路67号', '茂名市茂南区油城六路2号', '肇庆市端州区城中路49号', '清远市清城区人民二路18号', '潮州市湘桥区枫春路1号', '揭阳市榕城区临江北路39号', '云浮市云城区世纪大道行政中心',
       '澳门特别行政区风顺堂区南湾湖景大马路131号','香港金钟添美道2号']
columns=[' ',]+places
distance_datas=[]
duration_datas=[]
for sindex,sname in enumerate(places):
    distance_oneline=[sname,]
    duration_oneline = [sname,]
    s = getxy(sname)
    for eindex,ename in enumerate(places):
        if ename==sname:
            distance_oneline.append(0)
            duration_oneline.append(0)
        else:

            e = getxy(ename)
            distance, duration = getdrive(s, e)
            try:
                distance=min(distance,last_dis_df.iloc[sindex,eindex+1])
                duration = min(duration, last_dur_df.iloc[sindex, eindex + 1])
            except:
                pass
            distance_oneline.append(distance)
            duration_oneline.append(duration)
    distance_datas.append(distance_oneline)
    duration_datas.append(duration_oneline)
    print(distance_oneline)
    print(duration_oneline)

['广州市越秀区府前路1号市政府大院', 0, 136441.0, 127163.0, 416821.0, 34164.0, 220999.0, 199319.0, 385989.0, 141655.0, 270929.0, 69751.0, 84670.0, 90632.0, 224934.0, 421182.0, 340961.0, 98409.0, 77212.0, 410403.0, 379232.0, 145008.0, 139480.0, 167384.0]
['广州市越秀区府前路1号市政府大院', 0, 107.88333333333334, 123.08333333333333, 306.46666666666664, 40.13333333333333, 174.85, 147.25, 266.26666666666665, 126.8, 200.51666666666668, 65.9, 91.55, 82.31666666666666, 173.68333333333334, 305.3, 246.08333333333334, 89.76666666666667, 74.35, 293.35, 267.23333333333335, 113.71666666666667, 128.48333333333332, 145.56666666666666]
['深圳市民中心c区', 135290.0, 0, 154288.0, 335930.0, 136833.0, 329192.0, 176358.0, 342375.0, 92215.0, 168095.0, 74346.0, 118236.0, 145558.0, 303278.0, 499526.0, 419305.0, 217884.0, 199347.0, 356248.0, 325077.0, 264483.0, 164247.0, 65777.0]
['深圳市民中心c区', 113.71666666666667, 0, 133.63333333333333, 251.66666666666666, 115.9, 241.75, 139.65, 237.7, 89.56666666666666, 137.03333333333333, 70.31666666666666, 104.68

In [6]:
dis_df=pd.DataFrame(distance_datas,columns=columns)
dur_df=pd.DataFrame(duration_datas,columns=columns)
dis_df.to_excel('distance.xlsx',index=None)
dur_df.to_excel('duration.xlsx',index=None)